In [ ]:
import json
import base64

from pathlib import Path
from kubernetes import client
from kubernetes.config.incluster_config import InClusterConfigLoader
from kubernetes.client import V1Secret, V1ObjectMeta

In [ ]:
root = Path("/")
sa = root/Path("var/run/secrets/kubernetes.io/serviceaccount")
ns = open(sa/"namespace", "r").read()

# load and set kubernetes config
loader = InClusterConfigLoader(
    token_filename=sa/"token",
    cert_filename=sa/"ca.crt",
)
loader.load_and_set()

v1 = client.CoreV1Api()

In [ ]:
# get the kaggle credentials
# we assume that there is a `kaggle.json` file under `~/.kaggle`
# see: https://www.kaggle.com/docs/api#authentication
with open(Path.home()/".kaggle"/"kaggle.json", "r") as f:
    kaggle_creds = json.load(f)

In [ ]:
kaggle_secret = V1Secret(
    api_version="v1",
    kind="Secret",
    metadata=V1ObjectMeta(name="kaggle-secret"),
    string_data=kaggle_creds
)

In [ ]:
v1.create_namespaced_secret(namespace=ns, body=kaggle_secret)